In [1]:
import os
os.getcwd()

'/home/zbzhao/text-kgs-table/components/ft_data_prep'

In [2]:
import sys, os
sys.path.append('/home/zbzhao/text-kgs-table')
import pandas as pd
from components.kgs.dataset_kgs.e2e_kg import *
from utils import visualize_knowledge_graph_interactive, KnowledgeGraph, visualize_knowledge_graph
from components.retriever.DocData import *

In [3]:
file_path = f'../../data/CPL/text'

# Reading the file line by line and storing each line in a list
lines = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        lines.append(line.strip())
lines[0]

'1.卢江平与刘维辉民间借贷纠纷一审民事判决书.docx###安徽省合肥市包河区人民法院\\n民事判决书\\n（2016）皖0111民初3584号\\n原告：卢江平，男，1975年5月24日出生，汉族，住浙江省东阳市，\\n委托代理人：张学奇，安徽里奇律师事务所律师。\\n被告：刘维辉，男，1977年3月22日出生，汉族，住安徽省合肥市包河区，\\n委托代理人：沈国松，北京盈科（合肥）律师事务所律师。\\n委托代理人：唐婷婷，北京盈科（合肥）律师事务所实习律师。\\n原告卢江平诉被告刘维辉民间借贷纠纷一案，本院于2016年3月29日立案受理。依法由审判员方业泉适用简易程序公开开庭进行了审理。原告卢江平的委托代理人张学奇，被告刘维辉的委托代理人沈国松到庭参加诉讼。本案现已审理终结。\\n原告卢江平诉称：被告自2013年10月起，以经营周转为由向原告借款。后原告分别于2013年10月10日向被告汇款630500元、于2014年4月21日向被告汇款291000元，合计921500元。2015年5月12日，因被告无法按期偿还借款，经双方对账，被告于当日向原告出具借条一张，载明欠原告707100元借款，限于2015年6月30日之前付清。还款到期后，被告拒绝偿还借款。请求法院判令被告向原告偿还借款707100元、利息22921。66元（从2015年7月1日起按同期银行6个月贷款利率计算至起诉之日，后续按照年息4。35%的标准计算至实际还款之日止），两项合计730021。66元；本案诉讼费用由被告承担。\\n被告刘维辉辩称：被告虽然出具了707100元的借条，但没有实际收到借款，原告也未提交转账凭证。请求法院驳回原告的全部诉讼请求。\\n经审理查明：被告刘维辉因缺少周转资金于2013年10月向原告卢江平提出借款，原告通过其配偶王来菊的银行卡于2013年10月10日转账支付被告630500元、于2014年4月21日转账支付被告291000元，合计921500元。经双方对账，被告于2015年5月12日向原告出具一份借条，内容为：今借到卢江平现金柒拾万零柒仟壹佰元整，此款限在2015年6月30日前付清。借条载明的还款期限届满后，被告没有归还借款。此后，原告催讨借款无果，遂于2016年3月29日向本院提起诉讼。\\n上述事实，有原告及其配偶王来菊的身份证复印件，原告结婚证复印件，借条，银

In [6]:
from components.kgs.dataset_kgs.e2e_rotowire_field import fields_list_rotowire, fields_list_e2e
from components.kgs.dataset_kgs.CPL_field import fields_list_CPL
import json
import numpy as np
import pandas as pd

In [7]:
ie_prompt_extract_value_template="""
你是一个有用的表格内容填充助手，可以根据用户提供的行值和列值以及其对应的原文段落，从原文中提取角色的属性字段值。

检查提供的段落是否包含对应角色的属性值。如果没有，回答'Bad Information'。
如果相关段落包含角色的属性值，按照给定的要求回答值。
按照下面提供的例子回答用户的问题：

角色: 原告
属性: 出借人（原告）需返回本金总额（元）
相关背景: 本案现已审理终结。原告卢江平诉称：被告自2013年10月起，以经营周转为由向原告借款。后原告分别于2013年10月10日向被告汇款630500元、于2014年4月21日向被告汇款291000元，合计921500元。2015年5月12日，因被告无法按期偿还借款，经双方对账，被告于当日向原告出具借条一张，载明欠原告707100元借款，限于2015年6月30日之前付清。还款到期后，被告拒绝偿还借款。请求法院判令被告向原告偿还借款707100元、利息22921。66元（从2015年7月1日起按同期银行6个月贷款利率计算至起诉之日，后续按照年息4。35%的标准计算至实际还款之日止），两项合计730021。66元；本案诉讼费用由被告承担。
角色选项范围: ['Team']
问题: 出借人（原告）需返回本金总额（元）的取值是什么?
回答: 707100

角色: 法院
属性: '法院1需返回逾期利息利息率类型'
相关背景: 被告刘维辉于本判决生效后十日内偿还原告卢江平借款本金707100元，并支付逾期利息（以707100元为基数，自2015年7月1日起按照同期银行六个月贷款利率计算至借款本金付清之日止）。
角色选项范围: ['1=年利率;', '2=年利息;', '3=月利率;', '4=月利息;', '5=日利率', '6=日利息', '7=中国人民银行同期同档次贷款基准利率', '8=贷款市场报价利率(LPR)', '9=中国人民银行同期同档次贷款基准利率的四倍', '10=贷款市场报价利率(LPR)的四倍', '11=同国家规定/司法保护上限', '12=借款额的占比', '13=约定的数额（固定数额）', '14=年固定分红', '15=月固定分红', '16=日固定分红']
问题: 法院1需返回逾期利息利息率类型的取值是什么?
回答: 7=中国人民银行同期同档次贷款基准利率

以下是用户的问题：

以下是用户的问题：
角色: {ROLE}
属性: {FIELD}
相关背景: {RELATED_CONTEXT}
值范围: {SCOPE}
问题: {FIELD}的取值是什么?
回答:
"""

In [9]:
import json
file_path = '../../data/CPL/table_processed.json'
# Load data from the JSON file
with open(file_path, 'r', encoding='utf-8') as file:
    table_data = json.load(file)

In [12]:
from components.kgs.dataset_kgs.CPL_kg import key_role_dict_naive

In [19]:
ft_data = []

i = 0

splitter = SpecialSubStringTextSplitter(SP_Str_List, "backward")
text = lines[i].replace("\\n", "\n")
segments_backward = splitter.split(text, 'backward')

#print("Backward Segments:")
#for segment in segments_backward:
#    print(f'--- Segment ---\n{segment}\n')

doc_store = DocStore(preprocessor=preprocessor,
                    doc_data = SplittedList2Doc(segments_backward))
doc_store.init_retriever()
doc_store.create_pipeline()

reference_table = table_data[i]

for query in reference_table.keys():
    try:
        query_docs = doc_store.retrieve(query,
                                    sparse_top_k = 5, 
                                    dense_top_k = 5, 
                                    join_top_k = 5, 
                                    reranker_topk = 1)[0].content
        role = key_role_dict_naive[query]['role']
        scope = key_role_dict_naive[query]['value_range']
        role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=query, SCOPE=scope, RELATED_CONTEXT=query_docs)
        role_value_answer = reference_table[query]
        if pd.isna(role_value_answer):
            role_value_answer = "No Info Found"
        ft_data.append({"instruction": "Given the context, extract the information", "input": role_value_prompt, "output": str(role_value_answer)})
    except Exception as e:
        print(f"Warning: {e}")

Updating Embedding:   0%|                                                                                          | 0/9 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:00, 338914.49 docs/s]                                                                                   


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
len(ft_data)

85

In [23]:
from tqdm import tqdm

ft_data = []

In [ ]:
splitter = SpecialSubStringTextSplitter(SP_Str_List, "backward")

for i in tqdm(range(len(lines))):
    text = lines[i].replace("\\n", "\n")
    segments_backward = splitter.split(text, 'backward')

    doc_store = DocStore(preprocessor=preprocessor,
                        doc_data = SplittedList2Doc(segments_backward))
    doc_store.init_retriever()
    doc_store.create_pipeline()

    reference_table = table_data[i]
    
    for query in reference_table.keys():
        try:
            query_docs = doc_store.retrieve(query,
                                        sparse_top_k = 5, 
                                        dense_top_k = 5, 
                                        join_top_k = 5, 
                                        reranker_topk = 1)[0].content
            role = key_role_dict_naive[query]['role']
            scope = key_role_dict_naive[query]['value_range']
            role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=query, SCOPE=scope, RELATED_CONTEXT=query_docs)
            role_value_answer = reference_table[query]
            if pd.isna(role_value_answer):
                role_value_answer = "No Info Found"
            ft_data.append({"instruction": "Given the context, extract the information", "input": role_value_prompt, "output": str(role_value_answer)})
        except Exception as e:
            print(f"Warning: {e}")

Preprocessing: 100%|██████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 3492.67docs/s]

Updating Embedding:   0%|                                                                                          | 0/9 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:00, 282701.72 docs/s]                                                                                   


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(ft_data)

In [ ]:
# Specify the filename
ft_data_path = './'#"/data/zhaozibo/text-kgs-table/LLaMA-Factory/data"
filename = 'cpl_ie_train.json'
file_path = os.path.join(ft_data_path, filename)
# Open the file in write mode
with open(file_path, 'w') as json_file:
    json.dump(ft_data, json_file)

In [12]:
from components.kgs.dataset_kgs.CPL_kg import *

In [ ]:
cpl_kg = KnowledgeGraph()

cpl_kg.add_entity('案件', case_value)
cpl_kg.add_entity('法院', {})
cpl_kg.add_entity('借款人（被告）', {})
cpl_kg.add_entity('担保人（被告）', {})
cpl_kg.add_entity('出借人（原告）', {})

cpl_kg.add_relationship('担保人（被告）', '借款人（被告）', {"借贷担保关系": 1})
cpl_kg.add_relationship('借款人（被告）', '担保人（被告）',  {"借贷担保关系": -1})
cpl_kg.add_relationship('出借人（原告）', '借款人（被告）', {"借贷关系": 1})
cpl_kg.add_relationship('借款人（被告）', '出借人（原告）',  {"借贷关系": -1})
cpl_kg.add_relationship('借款人（被告）', '案件',  {"Dummy": -1})
cpl_kg.add_relationship('案件',  '借款人（被告）', {"Dummy": -1})
cpl_kg.add_relationship('借款人（被告）', '案件',  {"Dummy": -1})
cpl_kg.add_relationship('案件',  '担保人（被告）', {"Dummy": -1})
cpl_kg.add_relationship('担保人（被告）', '案件',  {"Dummy": -1})
cpl_kg.add_relationship('案件',  '出借人（原告）', {"Dummy": -1})
cpl_kg.add_relationship('出借人（原告）', '案件',  {"Dummy": -1})
cpl_kg.add_relationship('案件',  '其他诉讼参与人', {"Dummy": -1})
cpl_kg.add_relationship('其他诉讼参与人', '案件',  {"Dummy": -1})

reference_table = table_data[0]
for query in reference_table.keys():
    role = key_role_dict_naive[query]['role']
    scope = key_role_dict_naive[query]['value_range']
    cpl_kg.update_entity(role, {query: None})
    



### Query Rewrite & Info Summarize based on KG

In [1]:
case_person_value

NameError: name 'case_person_value' is not defined

In [15]:
# visualize_knowledge_graph_interactive(cpl_kg)

In [17]:
# visualize_knowledge_graph(cpl_kg)